In [1]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arazs\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# import stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint

import numpy as np
import torch

## Load model

In [20]:
# Load model
from models import InferSent
model_version = 1
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [21]:
# Keep it on CPU or put it on GPU
use_cuda = True
model = model.cuda() if use_cuda else model

In [22]:
# If infersent1 -> use GloVe embeddings. If infersent2 -> use InferSent embeddings.
W2V_PATH = 'GloVe/glove.840B.300d.txt' if model_version == 1 else 'fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

In [23]:
# Load embeddings of K most frequent words
model.build_vocab_k_words(K=100000)

Vocab size : 100000


### Updating Model with Vocab from our Dataset

In [25]:
all_para_sentences = []
for para in data:
    all_para_sentences.extend(para)

In [27]:
len(all_para_sentences)

6575

In [28]:
model.update_vocab(all_para_sentences)

Found 1401(/1972) words with w2v vectors
New vocab size : 101816 (added 1401 words)


In [29]:
all_q_sentences = []
for para in data_qs:
    all_q_sentences.extend(para)

In [31]:
model.update_vocab(all_q_sentences)

Found 121(/293) words with w2v vectors
New vocab size : 101937 (added 121 words)


## Load sentences

### Trying with MultiRC Dataset

In [7]:

f = open("Paragraphs.json","r",encoding="utf8")
data = json.load(f)
print(len(data))

456


In [8]:

f = open("Questions.json","r",encoding="utf8")
data_qs = json.load(f)
print(len(data_qs))

456


In [32]:
Para_Embeddings = []

In [33]:
for para in data:
  para_embed = model.encode(para)
  print((para_embed))
  Para_Embeddings.append(para_embed)

C:\Users\arazs\Documents\GitHub\InferSent\models.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  sentences = np.array(sentences)[idx_sort]


[[ 0.11256243  0.09040198  0.0365129  ...  0.          0.
   0.        ]
 [ 0.08737183  0.1160657   0.11471046 ...  0.00090266  0.
   0.01330047]
 [ 0.0961346   0.07750011  0.13451654 ...  0.          0.
   0.00076486]
 ...
 [ 0.04519949  0.02378928  0.00222661 ...  0.          0.
   0.        ]
 [ 0.0420932   0.07437453  0.         ...  0.          0.
   0.01302759]
 [ 0.1645855   0.11042652  0.12777373 ...  0.06823937 -0.03814263
  -0.01091907]]
[[ 0.13017015  0.12825833  0.10208301 ...  0.0449623   0.
   0.01596533]
 [ 0.1303148   0.19024482  0.13078132 ...  0.0609626   0.
   0.03200877]
 [ 0.11340217  0.13293453  0.05816589 ...  0.11457216  0.
   0.02241952]
 ...
 [ 0.14254619  0.20428835  0.10164339 ...  0.11937881 -0.03814263
   0.07896413]
 [ 0.15406513  0.19931452  0.12465417 ...  0.09522928  0.
   0.05935117]
 [ 0.04341318  0.09296721  0.06162942 ...  0.01124588  0.
   0.04815275]]
[[0.08997444 0.18418273 0.06180476 ... 0.         0.         0.10357223]
 [0.01546932 0.11675488

[[0.08195855 0.07108551 0.09474102 ... 0.         0.         0.15524845]
 [0.13404034 0.10306878 0.16485506 ... 0.         0.         0.05618284]
 [0.12369319 0.11787507 0.320627   ... 0.05428101 0.         0.        ]
 ...
 [0.12483554 0.         0.06866399 ... 0.         0.         0.03563702]
 [0.02964286 0.12692337 0.0333989  ... 0.         0.00373779 0.        ]
 [0.06885618 0.08767831 0.02414497 ... 0.05842281 0.         0.08982687]]
[[0.0824357  0.05452994 0.02569418 ... 0.02152194 0.         0.14892206]
 [0.05929901 0.         0.         ... 0.         0.         0.        ]
 [0.         0.08309527 0.19192606 ... 0.09395886 0.         0.07757204]
 ...
 [0.10973963 0.09264307 0.09220994 ... 0.02979544 0.0096184  0.1109168 ]
 [0.03388035 0.12541609 0.02506596 ... 0.         0.         0.07305411]
 [0.04823475 0.18017349 0.06021751 ... 0.21062861 0.         0.00423823]]
[[0.04242317 0.09951209 0.08473574 ... 0.08135092 0.         0.06275687]
 [0.06884048 0.         0.14464171 ... 

[[0.0182737  0.05433253 0.06206239 ... 0.         0.         0.08833628]
 [0.08054427 0.15733007 0.13426985 ... 0.20121425 0.         0.12107975]
 [0.02716731 0.02347423 0.03293387 ... 0.         0.         0.07709485]
 ...
 [0.12408795 0.06922981 0.17321563 ... 0.         0.         0.00937811]
 [0.07775933 0.07484798 0.05292077 ... 0.01336982 0.         0.        ]
 [0.05416057 0.05709263 0.12223606 ... 0.00119007 0.02686724 0.12866884]]
[[ 0.11404397  0.23324063  0.11610592 ...  0.08540387  0.
   0.05113541]
 [ 0.06479701  0.10649399  0.14518794 ...  0.05385767 -0.03814263
   0.15040427]
 [ 0.0818991   0.12602086  0.0429122  ...  0.08016574  0.
   0.        ]
 ...
 [ 0.15096235  0.18747497  0.12471325 ...  0.10806747  0.06980749
   0.        ]
 [ 0.04024624  0.04571805  0.0694982  ...  0.          0.
   0.        ]
 [ 0.0721503   0.12514041  0.0669819  ...  0.11275563  0.
   0.08429978]]
[[ 0.07362642  0.12261932  0.06068937 ...  0.03406104  0.
   0.07017533]
 [ 0.08824945  0.109688

[[0.07883009 0.16289473 0.09561199 ... 0.01632869 0.         0.06902518]
 [0.13348462 0.07789117 0.06581316 ... 0.04866718 0.0453878  0.        ]
 [0.07283948 0.13065973 0.04956153 ... 0.00495011 0.08094209 0.08572941]
 ...
 [0.06314491 0.18775699 0.05072599 ... 0.02383985 0.06266391 0.09189229]
 [0.05987185 0.22385381 0.02969334 ... 0.17795795 0.         0.05598975]
 [0.09863328 0.17123318 0.14000155 ... 0.13185214 0.06840011 0.00549719]]
[[0.04548664 0.19215782 0.1709632  ... 0.09300804 0.         0.09758569]
 [0.05308564 0.13722278 0.01740665 ... 0.05711279 0.         0.0827262 ]
 [0.07127562 0.09699831 0.06204746 ... 0.08570855 0.         0.10796627]
 [0.04184123 0.06605733 0.09859183 ... 0.02549334 0.         0.17152682]
 [0.02590156 0.12071372 0.02147545 ... 0.08540387 0.         0.1087042 ]
 [0.0726032  0.12545788 0.10966774 ... 0.09669779 0.01912871 0.13087735]]
[[0.         0.         0.00640664 ... 0.         0.         0.        ]
 [0.04756519 0.04924486 0.10496488 ... 0.020

...

[[ 0.05561338  0.05262516  0.10816484 ...  0.06642041  0.
   0.01112179]
 [ 0.06144081  0.17571536  0.176512   ...  0.04806753 -0.03814263
   0.01866371]
 [ 0.03911031  0.11970982  0.1654172  ...  0.11789382  0.
   0.04096813]
 ...
 [ 0.04065568  0.11191908  0.23017581 ...  0.11045273  0.
   0.08690103]
 [ 0.04471036  0.14840789  0.11455336 ...  0.08404306  0.
   0.02397155]
 [ 0.11263941  0.0903004   0.17777105 ...  0.09569976  0.
   0.        ]]
[[ 0.05993355  0.12392569  0.147448   ...  0.04153068  0.
   0.10165834]
 [ 0.08064657  0.11924526  0.15286708 ...  0.08687355  0.
   0.        ]
 [ 0.07163597  0.07812813  0.19374567 ...  0.05039162  0.
   0.        ]
 ...
 [ 0.06199264  0.11283135  0.07288831 ...  0.05539682  0.
   0.03141887]
 [ 0.06594267  0.03984115  0.0480254  ...  0.08182904  0.
   0.12793916]
 [ 0.06611731  0.14067459  0.10769299 ...  0.04021078 -0.03814263
   0.07242028]]
[[0.12112408 0.16098742 0.17425781 ... 0.01187832 0.         0.04158071]
 [0.02926079 0.077186  

[[ 0.09945538  0.09084766  0.1004021  ...  0.05895413  0.
   0.11914801]
 [ 0.13343984  0.09083916  0.16458474 ...  0.02137035  0.
   0.02543291]
 [ 0.06812619  0.03907299  0.0427472  ...  0.          0.
   0.02423102]
 ...
 [ 0.07600529  0.13988093  0.04892018 ...  0.0433143   0.
   0.04623434]
 [ 0.06688736  0.09255444  0.08727447 ... -0.0058067  -0.03814263
  -0.00260974]
 [ 0.07280605  0.1519391   0.00352008 ...  0.05165899  0.
   0.07475612]]
[[ 0.11295761  0.07226086  0.02370101 ...  0.01778534  0.00213842
  -0.0023117 ]
 [ 0.04102622  0.          0.05310618 ...  0.          0.
   0.        ]
 [ 0.04509727  0.          0.03029813 ...  0.          0.
   0.        ]
 ...
 [ 0.11332565  0.11935278  0.05459769 ...  0.          0.
   0.        ]
 [ 0.11402889  0.1296075   0.09298877 ...  0.00969966  0.
   0.        ]
 [ 0.05255849  0.11842152  0.03410666 ...  0.03952979 -0.03814263
  -0.02892261]]
[[ 0.09932129  0.11208988  0.17278774 ...  0.09240451  0.
   0.07553959]
 [ 0.06032992  

[[0.03793045 0.08682681 0.08901972 ... 0.03446769 0.         0.10349153]
 [0.02673325 0.07992407 0.08748668 ... 0.00187539 0.         0.02566   ]
 [0.09263825 0.0770428  0.07573321 ... 0.02224947 0.         0.02097924]
 ...
 [0.06969931 0.12152352 0.08123847 ... 0.03185687 0.         0.01080466]
 [0.06945748 0.0776144  0.02764826 ... 0.01861337 0.         0.13698232]
 [0.10845853 0.1699786  0.06751399 ... 0.13901927 0.         0.06181168]]
[[ 0.1106954   0.08386041  0.14247519 ...  0.01625244  0.
   0.04649177]
 [ 0.06458858  0.05956411  0.07210727 ...  0.02052599  0.
   0.        ]
 [ 0.0622832   0.10056836  0.07399541 ...  0.00545937  0.01829442
   0.03494282]
 ...
 [ 0.08831337  0.11028055  0.08760551 ...  0.03053238 -0.03814263
   0.01975989]
 [ 0.0617454   0.08574567  0.09639127 ...  0.06349754  0.
   0.08175956]
 [ 0.10821463  0.08173338  0.09240617 ...  0.07521322  0.
   0.04987996]]
[[0.08415671 0.15361328 0.08089235 ... 0.         0.         0.08867638]
 [0.04434554 0.15707144

[[0.12392396 0.11880558 0.12190828 ... 0.1120991  0.         0.09380836]
 [0.06118733 0.14537719 0.08816288 ... 0.00444017 0.         0.1621245 ]
 [0.11029758 0.13086055 0.07701958 ... 0.08804632 0.00227666 0.18870182]
 ...
 [0.07097587 0.13994098 0.0931429  ... 0.03783464 0.         0.09264551]
 [0.04415916 0.06713407 0.1014476  ... 0.08705615 0.         0.15970312]
 [0.01806481 0.0613388  0.06986698 ... 0.         0.         0.06734348]]
[[ 4.37906012e-02  1.55550569e-01  1.19916104e-01 ...  1.22907661e-01
  -3.81426290e-02  5.69200926e-02]
 [ 8.22899416e-02  9.01701823e-02  1.31060898e-01 ...  4.26522605e-02
   0.00000000e+00  6.03013001e-02]
 [ 6.29884526e-02  1.86794370e-01  1.17495142e-01 ...  2.08549067e-01
   0.00000000e+00  1.41395047e-01]
 ...
 [ 1.76530462e-02  6.77386746e-02  1.76944988e-04 ...  0.00000000e+00
   0.00000000e+00  1.26175165e-01]
 [ 2.15339139e-02  5.64740337e-02  4.90193851e-02 ...  0.00000000e+00
   0.00000000e+00  5.78501113e-02]
 [ 1.19557399e-02  1.54545

[[0.10107452 0.13827176 0.06173208 ... 0.00811141 0.         0.        ]
 [0.13318434 0.1267352  0.03514432 ... 0.04921711 0.         0.00600229]
 [0.17701821 0.14027941 0.04177975 ... 0.00848623 0.         0.05774042]
 ...
 [0.14485693 0.06352241 0.04966418 ... 0.03114488 0.         0.02096631]
 [0.14082766 0.12605351 0.05284642 ... 0.         0.         0.08847227]
 [0.13257931 0.13260937 0.0745487  ... 0.03331875 0.         0.        ]]
[[0.06704247 0.00959159 0.12021411 ... 0.         0.         0.03134554]
 [0.05724006 0.13433735 0.24181752 ... 0.13105579 0.01935864 0.05756753]
 [0.07231746 0.05711694 0.13992158 ... 0.         0.         0.02794782]
 ...
 [0.         0.04153308 0.00275494 ... 0.         0.         0.00805364]
 [0.04430659 0.04306128 0.02300135 ... 0.         0.         0.        ]
 [0.         0.         0.05816764 ... 0.         0.         0.00202363]]
[[ 0.12948716  0.13279553  0.14255288 ...  0.06134102  0.
   0.00387383]
 [ 0.09245121  0.16415879  0.12082583 .

In [34]:
print(len(Para_Embeddings))

456


In [35]:
Paragraph_Embeddings = np.array(Para_Embeddings)

C:\Users\arazs\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [36]:
np.save('Para_Embeddings_Infersent3.npy', Paragraph_Embeddings)

In [37]:
check_array = np.load('Para_Embeddings_Infersent3.npy', allow_pickle=True)

In [38]:
print(check_array[1].shape)

(14, 4096)


In [39]:
Q_Embeddings = []

for para in data_qs:
  para_embed = model.encode(para)
  print((para_embed))
  Q_Embeddings.append(para_embed)




[[0.06627975 0.04655779 0.07002031 ... 0.0425627  0.         0.06501141]
 [0.11186916 0.07775531 0.07010239 ... 0.         0.         0.01484449]
 [0.09156413 0.12611075 0.05272425 ... 0.01422109 0.         0.06497912]
 ...
 [0.14173077 0.1225329  0.14800398 ... 0.         0.         0.04248734]
 [0.08382923 0.17260209 0.08855174 ... 0.25468647 0.         0.02992734]
 [0.02504111 0.04743883 0.01561173 ... 0.         0.         0.01976585]]
[[0.08066359 0.1353543  0.05852262 ... 0.11620369 0.         0.        ]
 [0.10301598 0.12922329 0.07888107 ... 0.         0.         0.00585315]
 [0.07775934 0.07566264 0.08255339 ... 0.00387252 0.         0.01561254]
 ...
 [0.07279599 0.12368789 0.03516949 ... 0.03919126 0.         0.        ]
 [0.06929786 0.17442983 0.07413388 ... 0.04054999 0.         0.        ]
 [0.16019036 0.20578589 0.10959235 ... 0.08507206 0.         0.03351867]]
[[ 0.01632037  0.02736228  0.         ...  0.          0.
   0.04724886]
 [ 0.09003712  0.0980235   0.07948296 .

[[ 0.03173562  0.09216751  0.         ...  0.          0.01434752
   0.01750803]
 [ 0.04458864  0.12869051  0.10070822 ...  0.          0.
   0.01984805]
 [ 0.04685447  0.11068521  0.04997679 ...  0.16684431  0.
   0.01264489]
 ...
 [ 0.06529259  0.02870159  0.05120734 ...  0.00971686  0.
   0.00100518]
 [ 0.06727946  0.09792174  0.16084677 ...  0.10339686  0.
   0.        ]
 [ 0.06979549  0.11192173  0.17690404 ... -0.02954776 -0.03814263
   0.04692173]]
[[ 0.02159104  0.          0.         ...  0.          0.
   0.0671047 ]
 [ 0.03837072  0.          0.         ...  0.          0.
   0.03671401]
 [ 0.01895716  0.05664489 -0.03044512 ... -0.00096264 -0.03814263
   0.10033928]
 [ 0.07114334  0.02387922  0.         ...  0.          0.
   0.09247257]]
[[ 0.13249025  0.10960729  0.06634167 ...  0.05228628  0.
   0.08142243]
 [ 0.04413019  0.12488658  0.12994532 ... -0.03814677 -0.02535659
   0.00412765]
 [ 0.04151847  0.03883175  0.00734278 ...  0.04538458  0.
   0.02564886]
 ...
 [ 0.15

[[ 0.01007904  0.0322711   0.1183934  ...  0.          0.
   0.05556179]
 [ 0.02026472  0.04168938  0.09457808 ...  0.          0.
   0.06697952]
 [ 0.06353511  0.01353062  0.10208019 ...  0.05456854 -0.03814263
   0.01911205]]
[[ 0.10145038  0.11161226  0.06899521 ...  0.          0.
   0.        ]
 [ 0.08053312  0.10631149  0.09951525 ...  0.          0.
   0.        ]
 [ 0.04217203  0.07279117  0.14400625 ...  0.01432546 -0.03814263
   0.02983982]]
[[ 0.03157219  0.13812606  0.         ...  0.          0.03023988
   0.06074409]
 [ 0.07329272  0.10669613  0.09011748 ...  0.          0.
   0.09363902]
 [ 0.13347553  0.07632892  0.11345405 ...  0.03969083 -0.03814263
   0.03759957]]
[[ 0.07809937  0.17128935  0.02821385 ...  0.16769075  0.
   0.00450148]
 [ 0.0832277   0.08421261  0.         ...  0.03088002  0.
   0.        ]
 [ 0.0617738   0.12686026  0.06269103 ...  0.0168167   0.05240979
   0.        ]
 ...
 [ 0.03343156  0.12744208  0.08605783 ...  0.08168688  0.
   0.00479644]
 [ 

[[ 0.05300374  0.05591983  0.         ...  0.03987539  0.
   0.05373681]
 [ 0.07667299  0.01976141  0.05938672 ...  0.          0.
   0.        ]
 [ 0.09003712  0.          0.         ...  0.          0.
   0.00287599]
 ...
 [ 0.07114334  0.08464637  0.07861197 ...  0.01123654 -0.03814263
  -0.00295824]
 [ 0.06250834  0.20072734  0.08457047 ...  0.19596282  0.
   0.04943905]
 [ 0.00940901  0.12412451  0.06859569 ...  0.02736828  0.
   0.01442275]]
[[0.07417694 0.14874792 0.09654512 ... 0.06299321 0.         0.00937413]
 [0.07256025 0.09971236 0.01580969 ... 0.05802655 0.         0.06150896]
 [0.01941251 0.11255644 0.03079727 ... 0.02640027 0.01396959 0.08744604]
 ...
 [0.01524751 0.         0.02077509 ... 0.09115916 0.         0.        ]
 [0.05962299 0.12360836 0.07291803 ... 0.078871   0.         0.03035315]
 [0.04342872 0.06613174 0.08894566 ... 0.03875257 0.         0.02539873]]
[[ 0.04982885  0.04058081  0.         ...  0.          0.
   0.00256221]
 [ 0.05344677  0.03990749  0.05

[[ 0.15150896  0.10291563  0.03500805 ...  0.          0.
   0.02359217]
 [ 0.07207908  0.09522096  0.14592089 ...  0.13398752  0.
   0.02296963]
 [ 0.09836628  0.06381019  0.07119966 ...  0.          0.
   0.        ]
 ...
 [ 0.17918319  0.08518815  0.07098196 ...  0.          0.
   0.03673914]
 [ 0.10243385  0.07660881  0.14322399 ...  0.09158746 -0.03814263
   0.01476945]
 [ 0.13987933  0.11013117  0.07684769 ...  0.          0.
   0.01551802]]
[[ 5.3327952e-02  9.5218092e-02  6.4017378e-02 ...  6.9535375e-03
   0.0000000e+00  1.5249219e-02]
 [ 9.4277514e-03  2.9145794e-02  1.3115865e-01 ...  0.0000000e+00
   0.0000000e+00  1.0489316e-02]
 [ 8.9370176e-02  5.1442657e-02  1.4026920e-01 ...  0.0000000e+00
   0.0000000e+00  1.2857854e-04]
 [ 4.9925152e-02  1.1461518e-01  5.0014466e-02 ... -3.9267469e-02
  -1.9920705e-02  3.2906212e-02]
 [ 4.4792347e-02  1.3729808e-01  6.0924899e-02 ...  1.3428076e-01
   0.0000000e+00  2.0374980e-02]
 [ 6.1692465e-02  4.8180696e-02  1.4276721e-02 ...  2

...

[[ 0.03637554  0.08851351  0.0516024  ...  0.          0.
   0.03880672]
 [ 0.05990421  0.06636301  0.05097708 ...  0.07103761 -0.03814263
   0.08075007]
 [ 0.14150773  0.08579043  0.02032562 ...  0.          0.
   0.10288829]
 ...
 [ 0.07114332  0.08806182  0.1116538  ...  0.05021518  0.
   0.19492368]
 [ 0.07114332  0.12645967  0.06265218 ...  0.00902551  0.
   0.00367806]
 [ 0.02696974  0.08730073  0.00034711 ...  0.          0.
   0.17102487]]
[[ 0.08686671  0.0752152   0.1041049  ...  0.          0.
   0.02564743]
 [ 0.07114333  0.17411616  0.05137211 ...  0.03084748  0.
   0.        ]
 [ 0.06988297  0.20359334  0.01297865 ...  0.09972361  0.
   0.01350969]
 [ 0.06272721  0.05079967  0.         ...  0.01338771  0.
   0.        ]
 [ 0.09003712  0.22906196  0.08204704 ... -0.00989484  0.00739159
   0.02872475]
 [ 0.02607578  0.19679809  0.01116144 ...  0.07156838  0.
   0.08651815]]
[[ 0.08932779  0.1573599   0.00225355 ...  0.          0.
   0.10464168]
 [ 0.075641    0.02048706  0

[[0.08770816 0.03398713 0.         ... 0.         0.         0.03828231]
 [0.00423099 0.05342282 0.07847358 ... 0.0360118  0.         0.02644795]
 [0.14585115 0.08826422 0.13375045 ... 0.06167447 0.         0.00198737]
 ...
 [0.09269529 0.09004247 0.0769484  ... 0.03169279 0.         0.1021404 ]
 [0.03656468 0.11790802 0.00373643 ... 0.01296579 0.         0.        ]
 [0.15450796 0.08771739 0.07146907 ... 0.00189198 0.         0.02742356]]
[[ 0.10259758  0.03281106  0.07464797 ...  0.06662626  0.07394259
   0.08175821]
 [ 0.08772116  0.04621708  0.09978564 ...  0.          0.
   0.02507327]
 [ 0.09003708  0.08715352  0.         ...  0.          0.
   0.06150896]
 ...
 [ 0.03343156  0.09290916  0.00775267 ...  0.          0.
   0.06653184]
 [ 0.03300137  0.02748258  0.         ...  0.          0.
   0.11596915]
 [ 0.09424239  0.02104906  0.09327177 ...  0.03366346 -0.03814263
   0.02921575]]
[[0.04148835 0.12478585 0.06683551 ... 0.02254524 0.         0.01512298]
 [0.10895157 0.11531456

[[ 0.05353539  0.09683458  0.02234122 ...  0.02943906  0.
   0.09413849]
 [ 0.07066973  0.11655799  0.13515593 ... -0.00612784 -0.03814263
   0.04095376]
 [ 0.01516614  0.10038099  0.01477407 ...  0.          0.
   0.01339893]
 [ 0.04848839  0.06791589  0.05016472 ...  0.02712856  0.00349842
   0.07549139]]
[[ 0.09156413  0.1117935   0.04754326 ...  0.          0.
   0.07842495]
 [ 0.10967306  0.03544617  0.03685849 ...  0.          0.
   0.0054926 ]
 [ 0.1066189   0.05801387  0.01757628 ...  0.          0.
   0.104883  ]
 ...
 [ 0.04648329  0.04418255  0.01589621 ...  0.          0.
   0.12143455]
 [ 0.07111893  0.02545537  0.07413802 ... -0.03926747  0.0054794
   0.11519741]
 [ 0.08804698  0.02111919  0.02150937 ...  0.01784675  0.
   0.00592178]]
[[ 0.08178151  0.05567956  0.09617513 ...  0.05883597  0.
   0.04096163]
 [ 0.09515554  0.161055    0.13924554 ... -0.0169606  -0.03814263
   0.01617771]
 [ 0.13838078  0.11570881  0.0903574  ...  0.06533137  0.
   0.01719976]
 [ 0.11223384

[[0.03421977 0.09342144 0.23584792 ... 0.         0.         0.17166698]
 [0.02213759 0.02931744 0.04278149 ... 0.         0.         0.13140386]
 [0.05017463 0.06384747 0.09437224 ... 0.         0.         0.        ]
 ...
 [0.09616977 0.         0.07879858 ... 0.04370721 0.         0.00990985]
 [0.06577723 0.04958779 0.00424515 ... 0.04413179 0.         0.02774111]
 [0.06539159 0.06229097 0.07341633 ... 0.         0.         0.03190583]]
[[0.0806599  0.05777797 0.00891979 ... 0.0803638  0.         0.06270818]
 [0.04683187 0.05557464 0.01270175 ... 0.         0.         0.0429078 ]
 [0.00055549 0.12571706 0.00332958 ... 0.0173787  0.00718118 0.01396539]
 ...
 [0.         0.13100274 0.         ... 0.         0.         0.1904484 ]
 [0.0297482  0.1418321  0.10328512 ... 0.         0.         0.05523448]
 [0.04014434 0.08966478 0.04154347 ... 0.01896987 0.         0.20150523]]
[[ 0.09246039  0.0490857   0.05448968 ...  0.03557255  0.
   0.04627628]
 [ 0.14250368  0.1090838   0.07564347 .

[[ 0.113887    0.          0.         ...  0.          0.
   0.06932382]
 [ 0.06456821  0.09522096  0.10951977 ...  0.07094729 -0.03814263
   0.01549171]
 [ 0.08804822  0.09315733  0.         ...  0.01135743  0.
   0.        ]
 ...
 [ 0.14496757  0.          0.1173123  ...  0.          0.
   0.02166515]
 [ 0.05632941  0.          0.09540622 ...  0.          0.
   0.00726371]
 [ 0.04428947  0.02867295  0.         ...  0.          0.
   0.        ]]
[[ 0.05208988  0.01924101  0.07240242 ... -0.03090075 -0.02894576
   0.13289425]
 [ 0.06708057  0.19233204  0.03818626 ...  0.06971421  0.02829693
   0.07551895]
 [ 0.03316654  0.13136932  0.01266835 ...  0.13965413  0.
   0.09951881]]
[[ 0.06077174  0.08449056  0.0342869  ...  0.05026966  0.
   0.0149966 ]
 [ 0.09230419  0.16267814 -0.01629366 ... -0.02054783  0.03472597
   0.00750871]
 [ 0.04540374  0.07106455  0.         ...  0.          0.
   0.03053641]
 [ 0.03559977  0.07926522  0.01562377 ...  0.03211706  0.
   0.02331045]]
[[ 0.040541

In [40]:
print(len(Q_Embeddings))
Question_Embeddings = np.array(Q_Embeddings)
np.save('Ques_Embeddings_Infersent3.npy', Question_Embeddings)

456


C:\Users\arazs\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [41]:
check_array2 = np.load('Ques_Embeddings_Infersent3.npy', allow_pickle=True)

In [42]:
print(check_array2[1].shape)

(12, 4096)
